In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [3]:
img_height, img_width = 250, 250

num_training_samples = 1114 + 1109 #these numbers will change when more data is added: 1114 = number of male training samples, 1113 = num female
num_validation_samples = 813
num_epochs = 50


In [4]:
# import necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np

import os


In [5]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [7]:
# download pre-trained model file (one-time download)
dwnld_link = "https://github.com/arunponnusamy/cvlib/releases/download/v0.2.0/gender_detection.model"
model_path = get_file("gender_detection.model", dwnld_link,
                     cache_subdir="pre-trained", cache_dir=os.getcwd())

104194048/104188200 [==============================] - 188s 2us/step


In [8]:
model = Sequential()

#Creating structure of our model
model.add(Conv2D(32, (3,3), input_shape = input_shape)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2), dim_ordering="th"))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten()) #converts 3D feature map to 1D feature vector
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('tanh'))

rmsprop = optimizers.RMSprop(lr=0.0003, rho=0.9, epsilon=1e-08, decay=0.0)
adam = optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss = 'binary_crossentropy', #doing binary classification of gender
              optimizer=adam,
              metrics=['accuracy']
              )

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


In [9]:
batch_size = 16

In [10]:
#configuring augmentations for training set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip = True
        )

In [11]:
#Same but with testing
test_datagen = ImageDataGenerator(rescale = 1./255)


In [12]:
from keras.callbacks import ModelCheckpoint


In [13]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [14]:
# serialize model to JSON
model_json = model.to_json()
with open("final_model.json", "w") as json_file:
    json_file.write(model_json)

In [15]:
print("Model saved to json file model.json")

Model saved to json file model.json


In [16]:
# save weights to file - to be used later
model.save_weights('final_weights.h5')


In [17]:
print("Weights saved to h5 file weights.h5")

Weights saved to h5 file weights.h5


In [18]:
pwd

'/home/nbuser/library'

### Models for image classification with weights trained on ImageNet
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* DenseNet
* NASNet